# Loop over satellite datafiles, making annotated images 

Annotation code from Claude, Sep 17 2025 
https://claude.ai/public/artifacts/2b14fb0a-0ee5-4590-ae5b-a9dc841e5970

Prompt 
You got it, I want the semi-transparent annotations to spread with time, conserving ink as if an amount had been added with an eyedropper at the event locations. Perhaps make room for a fading time scale,  beyond that spatial spreading effect, in case the accumulation gets too thick, although perhaps it will not since the hypothetical circles over which the 'ink' is spread can expand beyond the image area. I want three annotation layers. Two are similar, let's call them brown and green. Each is an expanding and widening circle, perhaps smoothed a bit at its perimeter, concentric on the event location where it was launched. The brown annulus moves 52 pixels per time step, the green one moves 23 pixels per time step. The third annotation is a little more complicated: for every event, select randomly from a list of dirtycolors = ['orange','red','pink'] and speeds = [10,7,4] pixels per time step. Unlike the brown and green annotations, which fade only in an ink-conserving way, and accumulate over time, these dirty circles erase all the other dirty colors and replace them with the new dirty color.


In [3]:
import numpy as np
import random
from collections import deque
import matplotlib.patches as patches
from matplotlib.image import imread

from pathlib import Path

In [2]:
class EyedropperAnnotator:
    def __init__(self):
        # Store active events: [(lon, lat, birth_time), ...] for brown/green
        # [(lon, lat, birth_time, color, speed), ...] for dirty
        self.brown_events = deque()
        self.green_events = deque()
        self.dirty_events = deque()
        self.dirtycolors = ['orange', 'red', 'pink']
        self.speeds = [10, 7, 4]
        
    def add_events(self, event_locations, current_time):
        """Add new events at current time"""
        for lon, lat in event_locations:
            # Add brown and green events
            self.brown_events.append((lon, lat, current_time))
            self.green_events.append((lon, lat, current_time))
            
            # Add random dirty event (clears previous dirty events)
            color = random.choice(self.dirtycolors)
            speed = random.choice(self.speeds)
            self.dirty_events.clear()  # Erase previous dirty colors
            self.dirty_events.append((lon, lat, current_time, color, speed))
    
    def _draw_ring(self, overlay, x, y, inner_r, outer_r, color, alpha):
        """Draw a smoothed ring with given transparency"""
        if outer_r <= 0:
            return
            
        # Create mask for the ring
        mask = np.zeros((self.h, self.w), dtype=np.uint8)
        if outer_r > 0:
            cv2.circle(mask, (int(x), int(y)), int(outer_r), 255, -1)
        if inner_r > 0:
            cv2.circle(mask, (int(x), int(y)), int(inner_r), 0, -1)
        
        # Smooth the edges
        mask = cv2.GaussianBlur(mask, (5, 5), 1.0)
        
        # Apply color with alpha blending
        mask_norm = mask.astype(np.float32) / 255.0 * alpha
        for c in range(3):
            overlay[:, :, c] = overlay[:, :, c] * (1 - mask_norm) + color[c] * mask_norm

    
    def plot_one_expanding_ring(self, ax, current_time):
        """Plot annotations directly on matplotlib axes"""
        import matplotlib.patches as patches

        print('events ', self.brown_events)
        # Brown annotation (52 m/s = 0.842 deg per half hour)
        for lon, lat, birth_time in list(self.brown_events):
            age = current_time - birth_time
            if age < 0:
                continue
                
            outer_r = age * 52 *60*30 /111111.1  # 52 m/s x 1000s converted to degrees
            
            # Ink-conserving alpha, area is in square degrees so 1/area or 2/area 
            area = np.pi * (outer_r**2)
            alpha = min(0.8, 5./ max(area, 1))
            
            if alpha > 0.01:
                # Draw Annulus https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Annulus.html
                if outer_r > 0:
                    annulus = patches.Annulus((lon, lat), outer_r, width=outer_r/2, 
                                          facecolor='saddlebrown', alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)
            else:
                self.brown_events.remove((lon, lat, birth_time))

            print('alpha, outer_r, area', alpha, outer_r, area)

In [6]:
# Define the folder path to the image files 

inpath = Path('/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/Images_raw/')
outpath = Path('/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/Images_annot/')

files = sorted(inpath.glob("*.png"))

In [7]:
file = files[0]
file.name

'frame_0000.png'

PosixPath('/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/Images_raw/frame_0000.png')

In [4]:
for it, file in enumerate(files): 
    
    img = imread(file) 
    
# code to detect or specify events
    # event_locations = detect_events(img)  # Returns [(x1,y1), (x2,y2), ...]
    event_locations = [(300 + 10*it,500)]  # one event per frame  
 
# Events drive expanding rings (annulus). Initiate with the image
    annotator = EyedropperAnnotator(img.shape)
    annotator.add_events(event_locations, it)
    annotator.plot_one_expanding_ring(ax, it)

    annotated_image = annotator.annotate_image(img, it)
    
    cv2.imwrite(f'frame_{t:04d}.png', annotated_image)
    
    # Save the image to disk
    output_filename = outpath + filename + '.png'
    plt.savefig(output_filename, dpi=150, bbox_inches='tight')
    
    # Close the figure to free memory
    plt.close(fig)   
    
    # Add eyedropper annotations
    annotator.add_events(event_locations, it)

    # Save the image to disk
    output_filename = outpath + filename + '.annot.png'
    #output_filename = output_path + filename + '.annot.png'
    plt.savefig(, dpi=150, bbox_inches='tight')        
    plt.close(fig)  
    
    print(output_filename)

events  deque([])
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/Images_annot/time000.nc.annot.png


KeyboardInterrupt: 